# Searching Unstructured and Structured Data #
## Assignment 1: Retrieval models [100 points] ##
**TA**: Nikos Voskarides (n.voskarides@uva.nl)

In this assignment you will get familiar with basic information retrieval concepts. You will implement and evaluate different information retrieval ranking models and evaluate their performance.

We provide you with a Indri index. To query the index, you'll use a Python package ([pyndri](https://github.com/cvangysel/pyndri)) that allows easy access to the underlying document statistics.

For evaluation you'll use the [TREC Eval](https://github.com/usnistgov/trec_eval) utility, provided by the National Institute of Standards and Technology of the United States. TREC Eval is the de facto standard way to compute Information Retrieval measures and is frequently referenced in scientific papers.

This is a **groups-of-three assignment**, the deadline is **Monday, 22/1, at 23:59**. Code quality, informative comments and convincing analysis of the results will be considered when grading. Submission should be done through blackboard, questions can be asked on the course [Piazza](https://piazza.com/class/ixoz63p156g1ts).

### Technicalities (must-read!) ###

The assignment directory is organized as follows:
   * `./assignment.ipynb` (this file): the description of the assignment.
   * `./index/`: the index we prepared for you.
   * `./ap_88_90/`: directory with ground-truth and evaluation sets:
      * `qrel_test`: test query relevance collection (**test set**).
      * `qrel_validation`: validation query relevance collection (**validation set**).
      * `topics_title`: semicolon-separated file with query identifiers and terms.

You will need the following software packages (tested with Python 3.5 inside [Anaconda](https://conda.io/docs/user-guide/install/index.html)):
   * Python 3.5 and Jupyter
   * Indri + Pyndri (Follow the installation instructions [here](https://github.com/nickvosk/pyndri/blob/master/README.md))
   * gensim [link](https://radimrehurek.com/gensim/install.html)
   * TREC Eval [link](https://github.com/usnistgov/trec_eval)

In [ ]:
import collections
import copy
import gensim
import gensim.models.keyedvectors as word2vec
import glob
import io
import logging
import numpy as np
import os
import pandas as pd
import pyndri
import pyndri.compat
import re
import sys
import time

from gensim.similarities import WmdSimilarity
from matplotlib import pyplot as plt
from scipy.stats import ttest_rel
from sklearn.metrics.pairwise import cosine_similarity
from itertools import combinations

plt.style.use('ggplot')

### Pyndri primer ###
For this assignment you will use [Pyndri](https://github.com/cvangysel/pyndri) [[1](https://arxiv.org/abs/1701.00749)], a python interface for [Indri](https://www.lemurproject.org/indri.php). We have indexed the document collection and you can query the index using Pyndri. We will start by giving you some examples of what Pyndri can do:

First we read the document collection index with Pyndri:

In [83]:
index = pyndri.Index('index/')
token2id, id2token, _ = index.get_dictionary()

In [3]:
def write_run(model_name, data, out_f,
              max_objects_per_query=sys.maxsize,
              skip_sorting=False):
    """
    Write a run to an output file.
    Parameters:
        - model_name: identifier of run.
        - data: dictionary mapping topic_id to object_assesments;
            object_assesments is an iterable (list or tuple) of
            (relevance, object_id) pairs.
            The object_assesments iterable is sorted by decreasing order.
        - out_f: output file stream.
        - max_objects_per_query: cut-off for number of objects per query.
    """
    for subject_id, object_assesments in data.items():
        if not object_assesments:
            logging.warning('Received empty ranking for %s; ignoring.',
                            subject_id)

            continue

        # Probe types, to make sure everything goes alright.
        # assert isinstance(object_assesments[0][0], float) or \
        #     isinstance(object_assesments[0][0], np.float32)
        assert isinstance(object_assesments[0][1], str) or \
            isinstance(object_assesments[0][1], bytes)

        if not skip_sorting:
            object_assesments = sorted(object_assesments, reverse=True)

        if max_objects_per_query < sys.maxsize:
            object_assesments = object_assesments[:max_objects_per_query]

        if isinstance(subject_id, bytes):
            subject_id = subject_id.decode('utf8')

        for rank, (relevance, object_id) in enumerate(object_assesments):
            if isinstance(object_id, bytes):
                object_id = object_id.decode('utf8')

            out_f.write(
                '{subject} Q0 {object} {rank} {relevance} '
                '{model_name}\n'.format(
                    subject=subject_id,
                    object=object_id,
                    rank=rank + 1,
                    relevance=relevance,
                    model_name=model_name))

### Parsing the query file
You can parse the query file (`ap_88_89/topics_title`) using the following snippet:

In [4]:
def parse_topics(file_or_files,
                 max_topics=sys.maxsize, delimiter=';'):
    assert max_topics >= 0 or max_topics is None

    topics = collections.OrderedDict()

    if not isinstance(file_or_files, list) and \
            not isinstance(file_or_files, tuple):
        if hasattr(file_or_files, '__iter__'):
            file_or_files = list(file_or_files)
        else:
            file_or_files = [file_or_files]

    for f in file_or_files:
        assert isinstance(f, io.IOBase)

        for line in f:
            assert(isinstance(line, str))

            line = line.strip()

            if not line:
                continue

            topic_id, terms = line.split(delimiter, 1)

            if topic_id in topics and (topics[topic_id] != terms):
                    logging.error('Duplicate topic "%s" (%s vs. %s).',
                                  topic_id,
                                  topics[topic_id],
                                  terms)

            topics[topic_id] = terms

            if max_topics > 0 and len(topics) >= max_topics:
                break

    return topics

# with open('./ap_88_89/topics_title', 'r') as f_topics:
#     print(parse_topics([f_topics]))

### Task 1: Implement and compare lexical IR methods [40 points] ### 

In this task you will implement a number of lexical methods for IR using the **Pyndri** framework. Then you will evaluate these methods on the dataset we have provided using **TREC Eval**.

Use the **Pyndri** framework to get statistics of the documents (term frequency, document frequency, collection frequency; **you are not allowed to use the query functionality of Pyndri**) and implement the following scoring methods in **Python**:

- [TF-IDF](http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html). **[5 points]**
- [BM25](http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html) with k1=1.2 and b=0.75. **[5 points]**
- Language models ([survey](https://drive.google.com/file/d/0B-zklbckv9CHc0c3b245UW90NE0/view))
    - Jelinek-Mercer (explore different values of 𝛌 in the range [0.1, 0.5, 0.9]). **[10 points]**
    - Dirichlet Prior (explore different values of 𝛍 [500, 1000, 1500]). **[5 points]**
    - Absolute discounting (explore different values of 𝛅 in the range [0.1, 0.5, 0.9]). **[5 points]**
    
Implement the above methods and report evaluation measures (on the test set) using the hyper parameter values you optimized on the validation set (also report the values of the hyper parameters). Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

For the language models, create plots showing `NDCG@10` with varying values of the parameters. You can do this by chaining small scripts using shell scripting (preferred) or execute trec_eval using Python's `subprocess`.

Compute significance of the results using a [two-tailed paired Student t-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html) **[10 points]**. Be wary of false rejection of the null hypothesis caused by the [multiple comparisons problem](https://en.wikipedia.org/wiki/Multiple_comparisons_problem). There are multiple ways to mitigate this problem and it is up to you to choose one.

Analyse the results by identifying specific queries where different methods succeed or fail and discuss possible reasons that cause these differences. This is *very important* in order to understand how the different retrieval functions behave.

**NOTE**: Don’t forget to use log computations in your calculations to avoid underflows. 

**IMPORTANT**: You should structure your code around the helper functions we provide below.

In [5]:
with open('./ap_88_89/topics_title', 'r') as f_topics:
    queries = parse_topics([f_topics])

index = pyndri.Index('index/')
start_time = time.time()
num_documents = index.maximum_document() - index.document_base()

dictionary = pyndri.extract_dictionary(index)

tokenized_queries = {
    query_id: [dictionary.translate_token(token)
               for token in index.tokenize(query_string)
               if dictionary.has_token(token)]
    for query_id, query_string in queries.items()}

query_term_ids = set(
    query_term_id
    for query_term_ids in tokenized_queries.values()
    for query_term_id in query_term_ids)

print('Gathering statistics about', len(query_term_ids), 'terms.')

# inverted index creation.

document_lengths = {}
unique_terms_per_document = {}

inverted_index = collections.defaultdict(dict)
collection_frequencies = collections.defaultdict(int)

total_terms = 0

for int_doc_id in range(index.document_base(), index.maximum_document()):
    ext_doc_id, doc_token_ids = index.document(int_doc_id)

    document_bow = collections.Counter(
        token_id for token_id in doc_token_ids
        if token_id > 0)
    document_length = sum(document_bow.values())

    document_lengths[int_doc_id] = document_length
    total_terms += document_length

    unique_terms_per_document[int_doc_id] = len(document_bow)

    for query_term_id in query_term_ids:
        assert query_term_id is not None

        document_term_frequency = document_bow.get(query_term_id, 0)

        if document_term_frequency == 0:
            continue

        collection_frequencies[query_term_id] += document_term_frequency
        inverted_index[query_term_id][int_doc_id] = document_term_frequency

avg_doc_length = total_terms / num_documents

print('Inverted index creation took', time.time() - start_time, 'seconds.')

Gathering statistics about 456 terms.
Inverted index creation took 37.81690216064453 seconds.


In [ ]:
def run_retrieval(model_name, score_fn, **args):
    """
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.
    
    :param model_name: the name of the model (a string)
    :param score_fn: the scoring function (a function - see below for an example) 
    """
    run_out_path = '{}.run'.format(model_name)

    if os.path.exists(run_out_path):
        return

    retrieval_start_time = time.time()

    print('Retrieving using', model_name)

    data = {}
    
    for qid in tokenized_queries.keys(): #for each query
#     for qid in ['93']: #test

        score = {}
            #hack for BM25 (iterate duplicate terms in query once)
        try:
            if score_fn==bm25:
                tokenized_queries[qid] = list(set(tokenized_queries[qid]))
        except:
            pass
        
        for query_term_id in tokenized_queries[qid]: #for each word_id in query

            for (int_document_id,document_term_freq) in zip(inverted_index[query_term_id].keys(),inverted_index[query_term_id].values()):
                    #for each document where the word appears, calculate scoring function increment its score
                new_score = score_fn(int_document_id, query_term_id, document_term_freq, **args)
                
                if int_document_id not in score.keys():
                    #print('initiallizing and incrementing score')
                    score[int_document_id] = new_score
                else:
                    #print('incrementing score')
                    score[int_document_id] += new_score

        #turn dict to list of tuples and continue        
        # The dictionary data should have the form: query_id --> (document_score, external_doc_id)
        data[qid] = [(score, index.ext_document_id(doc_id)) for (score, doc_id) in zip(score.values(), score.keys())]
                
#     write out
    with open(run_out_path, 'w') as f_out:
        write_run(
            model_name=model_name,
            data=data,
            out_f=f_out,

            max_objects_per_query=1000)

### TF-IDF

In [92]:
def tfidf(int_document_id, query_term_id, document_term_freq):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term 
    """   
    
    ## Amount of time a term appears in a document  
    df = document_term_freq
    ## Total terms in a document
    dl = document_lengths[int_document_id]
    ## Amount of documents containing a term
    n = len(inverted_index[query_term_id])
    
    tf = np.log2(1 + df)
    idf = np.log2((n/df))
    
    score = tf * idf

    return score

In [ ]:
start_time = time.time()

run_retrieval('retrievals/tf-idf',tfidf)

print ('elapsed time:',time.time()-start_time)

In [ ]:
# trec_eval command for evalutaion of tf-idf 
!trec_eval -m all_trec ./ap_88_89/qrel_validation retrievals/tf-idf.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"

# trec_eval command for test of tf-idf 
!trec_eval -m all_trec ./ap_88_89/qrel_test retrievals/tf-idf.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"

### BM25

In [ ]:
for q_id,q in zip(tokenized_queries.keys() ,tokenized_queries.values()):
    if len(q) != len(set(q)):
        print('query %s has duplicate words!'%q_id)

In [ ]:
def bm25(int_document_id, query_term_id, document_term_freq ,k1=1.2 , b=0.75,length_avg = round(np.mean(np.array(list(document_lengths.values())))) ):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term
    
    Hyperparameters: k1=1.2 and b=0.75
    
    """
    
    ## Amount of time a term appears in a document  
    df = document_term_freq
    ## Total terms in a document
    dl = document_lengths[int_document_id]
    ## Amount of documents containing a term
    n = len(inverted_index[query_term_id])
    
    tf = np.log2(1 + df)
    idf = np.log2((n/df))

    score = ((k1+1)*tf*idf)/(k1*((1-b)+b*(document_lengths[int_document_id] /length_avg))+tf)
    
    return score

In [ ]:
start_time=time.time() 

run_retrieval('retrievals/bm25',bm25)

print('BM25 scoring took', time.time() - start_time, 'seconds.')

In [ ]:
# trec_eval command for evalutaion of bm25
!trec_eval -m all_trec ./ap_88_89/qrel_validation retrievals/bm25.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"

# trec_eval command for test of bm25
!trec_eval -m all_trec ./ap_88_89/qrel_test retrievals/bm25.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"

### Jelinek-Mercer

In [ ]:
# #precompute the 2nd term of jelinek mercer smoothening to speed-up computations
# P_mle_col = dict()
# for query_term_id in query_term_ids:
#     P_mle_col[query_term_id] = index.get_term_frequencies()[query_term_id] / index.total_terms()

#precompute the logged 2nd term of jelinek mercer smoothening to speed-up computations
P_mle_col_log = dict()
for query_term_id in query_term_ids:
    P_mle_col_log[query_term_id] = np.log(index.get_term_frequencies()[query_term_id]) - np.log(index.total_terms())

In [ ]:
def jelinek_mercer(int_document_id, query_term_id, document_term_freq, lambd = 0.75):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term
    :param lambd: lambda value of linear interpolation (default = 0.75)
    """
    
    return np.logaddexp( (np.log(lambd) + np.log(document_term_freq) - np.log(document_lengths[int_document_id])),
                        (np.log(1-lambd) + P_mle_col_log[query_term_id]) )

In [ ]:
start_time = time.time()

test_dict = run_retrieval('retrievals/jelinek_mercer',jelinek_mercer)

print ('elapsed time:',time.time()-start_time)

In [ ]:
# trec_eval command for evalutaion of jelinek_mercer
!trec_eval -m all_trec ./ap_88_89/qrel_validation retrievals/jelinek_mercer.run | grep -E "^ndcg_cut_10\s|^map\s|^P_5\s|^recall_1000\s"

# trec_eval command for test of jelinek_mercer
!trec_eval -m all_trec ./ap_88_89/qrel_test retrievals/jelinek_mercer.run | grep -E "^ndcg_cut_10\s|^map\s|^P_5\s|^recall_1000\s"


### Dirichlet Prior

In [ ]:
def dirichlet_prior(int_document_id, query_term_id, document_term_freq, mu=500):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term 
    """
    
    ## Amount of time a term appears in a document  
    df = document_term_freq
    
    tf = np.log2(1 + df)
    
#     return np.logaddexp( (np.log(document_term_frequency)) ,
#                         (np.log(mu) + P_mle_col_log[query_term_id]) ) - np.log( document_lengths[int_document_id] + mu )

    return np.log(tf+mu*P_mle_col_log[query_term_id])-np.log(document_lengths[int_document_id]+mu)


In [ ]:
start_time = time.time()

test_dict = run_retrieval('retrievals/dirichlet_prior',dirichlet_prior)

print ('elapsed time:',time.time()-start_time)

In [ ]:
# #trec_eval command for evalutaion of dirichlet_prior
!trec_eval -m all_trec ./ap_88_89/qrel_validation retrievals/dirichlet_prior.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"


# #trec_eval command for test of dirichlet_prior
!trec_eval -m all_trec ./ap_88_89/qrel_test retrievals/dirichlet_prior.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"


### Absolute discounting

In [ ]:
def absolute_discounting(int_document_id, query_term_id, document_term_freq, delta=0.1):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term 
    """

    return np.logaddexp(np.log( max( document_term_frequency - delta ,0 ) ) - np.log( document_lengths[int_document_id] ) ,
                              np.log(delta * unique_terms_per_document[int_doc_id] ) + P_mle_col_log[query_term_id] - np.log( document_lengths[int_document_id] ) )
#     return np.log((np.max(tf-delta,1e-15)/doc_length)+
#             ((delta*collection_frequencies[query_term_id]*P_mle_col[query_term_id])/doc_length))

In [ ]:
start_time = time.time()

test_dict = run_retrieval('retrievals/absolute_discounting', absolute_discounting)

print ('elapsed time:',time.time()-start_time)

In [ ]:
# trec_eval command for evalutaion of absolute
!trec_eval -m all_trec ./ap_88_89/qrel_validation retrievals/absolute_discounting.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"

# trec_eval command for test of absolute
!trec_eval -m all_trec ./ap_88_89/qrel_test retrievals/absolute_discounting.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"

## Hyper-parameter testing¶

In [ ]:
def get_scores(filenames, validation=False):
    """
    Function to get scores from trec_eval using a 
    :param filenames: a list of *.run files (retrieval scores for queries)
    :param validation: boolean. Perform evaluation on validation or test set (queries)
    """
    scores = {}
    for filename in filenames:
        param_value = float(filename.split("=")[-1].split('.run')[0])
        if validation:
            result = !trec_eval -m all_trec ./ap_88_89/qrel_validation {filename} | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"
        else:
            result = !trec_eval -m all_trec ./ap_88_89/qrel_test {filename} | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"

        for row in result:
            metric = row.split(' ')[0]
            score = float(row.split('\t')[-1])
            if metric not in scores.keys():
                scores[metric] = {}
            scores[metric][param_value] = score
    return scores

In [ ]:
def test_hyperparameters(hyperparameters,funct_name,score_function):
    
    for param_value in hyperparameters:
        
        start_time = time.time()
        
        filename = 'retrievals/'+funct_name+'='+str(param_value)
        
        if funct_name== 'jelinek_mercer_lambda':
            run_retrieval(filename,score_function, lambd = param_value)
        elif funct_name=='dirichlet_prior_mu':
            run_retrieval(filename,dirichlet_prior, mu = param_value)
        
        elif funct_name=='absolute_discounting_delta':
            run_retrieval(filename,absolute_discounting, delta = param_value)
            
        else:
            print ("Wrong method name")
            return
            

        print ('finished {} in {} sec'.format(filename,(time.time()-start_time)))
    
    filenames = glob.glob('./retrievals/'+funct_name+'=*')
    scores = get_scores(filenames)
    scores = pd.DataFrame.from_dict(scores,dtype=float)
    scores.index.name = 'delta'
    
    return scores

In [ ]:
scores_jm = test_hyperparameters(np.arange(0.1,0.91,0.1),'jelinek_mercer_lambda',jelinek_mercer)
scores_jm.drop('recall_1000',axis=1).plot()

In [ ]:
scores_dirich = test_hyperparameters(np.arange(500,1501, 100),'dirichlet_prior_mu',dirichlet_prior)
scores_dirich.drop('recall_1000',axis=1).plot()

In [ ]:
scores_abs_disc = test_hyperparameters(np.arange(0.1,0.91, 0.1),'absolute_discounting_delta',absolute_discounting)
scores_abs_disc.drop('recall_1000',axis=1).plot()

## Significance testing

In [ ]:
ndcg_all={}

In [ ]:
columns = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

ndcg = pd.DataFrame(columns=columns)

for param_value in columns:
    start_time = time.time()

    filename = 'retrievals/jelinek_mercer_lambda='+str(param_value)

    run_retrieval(filename,jelinek_mercer, lambd = param_value)

    print ('finished {} in {} sec'.format(filename,(time.time()-start_time)))

    results = !trec_eval -m all_trec -q ./ap_88_89/qrel_test { filename }.run | grep -E "^ndcg_cut_10\s"

    for result in results:
        result = re.sub( '\s+', ' ', result ).strip().split()

        idx = result[1]
        value = result[2]
        param_value = round(param_value,1)

        ndcg.set_value(idx, param_value, value)
        
ndcg_all['jelinek_mercer'] = ndcg

In [ ]:
columns = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

ndcg = pd.DataFrame(columns=columns)

for param_value in columns:
    start_time = time.time()

    filename = 'retrievals/absolute_discounting_delta='+str(param_value)

    run_retrieval(filename,jelinek_mercer, lambd = param_value)

    print ('finished {} in {} sec'.format(filename,(time.time()-start_time)))

    results = !trec_eval -m all_trec -q ./ap_88_89/qrel_test { filename }.run | grep -E "^ndcg_cut_10\s"

    for result in results:
        result = re.sub( '\s+', ' ', result ).strip().split()

        idx = result[1]
        value = result[2]
        param_value = round(param_value,1)

        ndcg.set_value(idx, param_value, value)

ndcg_all['absolute_discounting'] = ndcg

In [ ]:
columns = np.arange(500,1501,100)

ndcg = pd.DataFrame(columns=columns)

for param_value in columns:
    start_time = time.time()

    filename = 'retrievals/dirichlet_prior_mu='+str(param_value)

    run_retrieval(filename,jelinek_mercer, lambd = param_value)

    print ('finished {} in {} sec'.format(filename,(time.time()-start_time)))

    results = !trec_eval -m all_trec -q ./ap_88_89/qrel_test { filename }.run | grep -E "^ndcg_cut_10\s"

    for result in results:
        result = re.sub( '\s+', ' ', result ).strip().split()

        idx = result[1]
        value = result[2]
        param_value = round(param_value,1)

        ndcg.set_value(idx, param_value, value)

ndcg_all['dirichlet_prior'] = ndcg

In [ ]:
tfidf_results = pd.DataFrame()

results = !trec_eval -m all_trec -q ./ap_88_89/qrel_test retrievals/tf-idf.run | grep -E "^ndcg_cut_10\s"

for result in results:
    result = re.sub( '\s+', ' ', result ).strip().split()

    index = result[1]
    value = float(result[2])

    tfidf_results.set_value(index, 1, value)

In [ ]:
bm25_results = pd.DataFrame()

results = !trec_eval -m all_trec -q ./ap_88_89/qrel_test retrievals/bm25.run | grep -E "^ndcg_cut_10\s"

for result in results:
    result = re.sub( '\s+', ' ', result ).strip().split()

    index = result[1]
    value = float(result[2])

    bm25_results.set_value(index, 1, value)

In [ ]:
best_scores = pd.DataFrame()
best_scores['absolute_discounting'] = ndcg_all['absolute_discounting'][0.1]
best_scores['jelinek_mercer'] = ndcg_all['jelinek_mercer'][0.1]
best_scores['dirichlet_prior'] = ndcg_all['dirichlet_prior'][1500]

best_scores['tf-idf'] = tfidf_results
best_scores['BM25'] = bm25_results


best_scores = best_scores.applymap(lambda x: float(x))

In [ ]:
#plot non-zero queries only
best_scores[best_scores.sum(axis=1)>0].plot(kind='bar',figsize=(18,8))

In [ ]:
def t_test(method1,method2,a=0.05):
    
    p_value = ttest_rel(method1, method2)[1]
    
    if p_value>a:
        return (p_value, 'not-rejected')
    else:
        return (p_value, 'rejected')

In [ ]:
combs = list(combinations(best_scores.columns,2))

for pair in combs:
    print(pair[0],pair[1])
    print(t_test(best_scores[pair[0]],best_scores[pair[1]],a=0.05/len(combs)))
    print('\n')

### Task 2: Latent Semantic Models (LSMs) [20 points] ###

In this task you will experiment with applying distributional semantics methods ([LSI](http://lsa3.colorado.edu/papers/JASIS.lsi.90.pdf) **[5 points]** and [LDA](https://www.cs.princeton.edu/~blei/papers/BleiNgJordan2003.pdf) **[5 points]**) for retrieval.

You do not need to implement LSI or LDA on your own. Instead, you can use [gensim](http://radimrehurek.com/gensim/index.html). An example on how to integrate Pyndri with Gensim for word2vec can be found [here](https://github.com/cvangysel/pyndri/blob/master/examples/word2vec.py). For the remaining latent vector space models, you will need to implement connector classes (such as `IndriSentences`) by yourself.

In order to use a latent semantic model for retrieval, you need to:
   * build a representation of the query **q**,
   * build a representation of the document **d**,
   * calculate the similarity between **q** and **d** (e.g., cosine similarity, KL-divergence).
     
The exact implementation here depends on the latent semantic model you are using. 
   
Each of these LSMs come with various hyperparameters to tune. Make a choice on the parameters, and explicitly mention the reasons that led you to these decisions. You can use the validation set to optimize hyper parameters you see fit; motivate your decisions. In addition, mention clearly how the query/document representations were constructed for each LSM and explain your choices.

In this experiment, you will first obtain an initial top-1000 ranking for each query using TF-IDF in **Task 1**, and then re-rank the documents using the LSMs. Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

Perform significance testing **[5 points]** (similar as in Task 1) in the class of semantic matching methods.

Perform analysis **[5 points]**

### Task 3:  Word embeddings for ranking [10 points] ###

First create word embeddings on the corpus we provided using [word2vec](http://arxiv.org/abs/1411.2738) -- [gensim implementation](https://radimrehurek.com/gensim/models/word2vec.html). You should extract the indexed documents using pyndri and provide them to gensim for training a model (see example [here](https://github.com/nickvosk/pyndri/blob/master/examples/word2vec.py)).

Try one of the following (increasingly complex) methods for building query and document representations:
   * Average or sum the word vectors.
   * Cluster words in the document using [k-means](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) and use the centroid of the most important cluster. Experiment with different values of K for k-means.
   * Using the [bag-of-word-embeddings representation](https://ciir-publications.cs.umass.edu/pub/web/getpdf.php?id=1248).
   
Note that since we provide the implementation for training word2vec, you will be graded based on your creativity on combining word embeddings for building query and document representations.

Note: If you want to experiment with pre-trained word embeddings on a different corpus, you can use the word embeddings we provide alongside the assignment (./data/reduced_vectors_google.txt). These are the [google word2vec word embeddings](https://code.google.com/archive/p/word2vec/), reduced to only the words that appear in the document collection we use in this assignment.

In [6]:
## Create embeddings
start_time = time.time()
print('Initializing word2vec.')

word2vec_init = gensim.models.Word2Vec(
    size=300,  # Embedding size
    window=5,  # One-sided window size
    sg=True,  # Skip-gram.
    min_count=1,  # Minimum word frequency.
    sample=1e-3,  # Sub-sample threshold.
    hs=False,  # Hierarchical softmax.
    negative=10,  # Number of negative examples.
    iter=1,  # Number of iterations.
    workers=8,  # Number of workers.
)

index = pyndri.Index('index/')

print('Loading vocabulary.')
dictionary = pyndri.extract_dictionary(index)
sentences = pyndri.compat.IndriSentences(index, dictionary)

print('Constructing word2vec vocabulary.')

# Build vocab.
word2vec_init.build_vocab(sentences, trim_rule=None)
model = word2vec_init
# Normalize vectors
model.init_sims(replace=True)

# models = [word2vec_init]

# for epoch in range(1, 1 + 1):
#     print('Epoch ', epoch)
#     epoch_time = time.time()
    
#     model = copy.deepcopy(models[-1])
#     model.train(sentences, total_examples = model.corpus_count, epochs=model.iter)

#     models.append(model)
#     print('Elapsed time for model', epoch, ':', time.time() - epoch_time)

# print('Trained models:', len(models))
print('Elapsed time:', time.time()-start_time)

Initializing word2vec.
Loading vocabulary.
Constructing word2vec vocabulary.
Elapsed time: 55.54707717895508


In [7]:
def run_w2v_retrieval(model_name, score_fn, **args):
    """
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.
    
    :param model_name: the name of the model (a string)
    :param score_fn: the scoring function (a function - see below for an example) 
    """
    run_out_path = '{}.run'.format(model_name)

    if os.path.exists(run_out_path):
        return

    retrieval_start_time = time.time()

    print('Retrieving using', model_name)

    data = {}
    
    for qid in tokenized_queries.keys(): #for each query
#     for qid in ['93']: #test

        score = {}
        
        for query_term_id in tokenized_queries[qid]: #for each word_id in query

            for (int_document_id,document_term_freq) in zip(inverted_index[query_term_id].keys(),inverted_index[query_term_id].values()):
                #for each document where the word appears, calculate scoring function increment its score
                new_score = score_fn(int_document_id, qid)
                
                if int_document_id not in score.keys():
                    #print('initiallizing and incrementing score')
                    score[int_document_id] = new_score
                else:
                    #print('incrementing score')
                    score[int_document_id] += new_score

        #turn dict to list of tuples and continue        
        # The dictionary data should have the form: query_id --> (document_score, external_doc_id)
        data[qid] = [(score, index.ext_document_id(doc_id)) for (score, doc_id) in zip(score.values(), score.keys())]
                
#     write out
    with open(run_out_path, 'w') as f_out:
        write_run(
            model_name=model_name,
            data=data,
            out_f=f_out,

            max_objects_per_query=1000)

In [ ]:
## Sum all query vectors (normalized)
summed_queries = {}
for qid in tokenized_queries.keys(): #for each query
    
    Q = []
    for query_term_id in tokenized_queries[qid]: #for each word_id in query
        term = dictionary[query_term_id]
        Q.append(model[term])
        
    Q = np.array(Q)
    vector = Q.sum(axis=0) 
#     vector = vector / np.sqrt((vector ** 2).sum())

    summed_queries[qid] = vector   

## Sum all document vectors (normalized)    
summed_docs = {}
for int_doc_id in range(index.document_base(), index.maximum_document()):
    ext_doc_id, doc_token_ids = index.document(int_doc_id)
        
    D = []
    for token_id in doc_token_ids: 
        if token_id > 0:
            term = dictionary[query_term_id]
            D.append(model[term])
    D = np.array(D)
    vector = D.sum(axis=0) 
#     vector = vector / np.sqrt((vector ** 2).sum())

    summed_docs[int_doc_id] = vector

## Sum of vectors

In [ ]:
def summed_vectors(document_id, query_id):
    """
    Scoring function for a document and a query term
    
    :param document_id: the document id
    :param query_id: the tokenized query id
    """
    
    cosine_similarity = np.dot(summed_queries[query_id], summed_docs[document_id])/(np.linalg.norm(summed_queries[query_id])* np.linalg.norm(summed_docs[document_id]))
        
    return cosine_similarity

In [ ]:
start_time = time.time()

run_w2v_retrieval('retrievals/summed_vectors',summed_vectors)

print ('elapsed time:',time.time()-start_time)

In [ ]:
# trec_eval command for evalutaion of summed_vectors
!trec_eval -m all_trec ./ap_88_89/qrel_validation retrievals/summed_vectors.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"
print('\n')

# trec_eval command for test of summed_vectors
!trec_eval -m all_trec ./ap_88_89/qrel_test retrievals/summed_vectors.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"

## Average of vectors

In [ ]:
def averaged_vectors(document_id, query_id):
    """
    Scoring function for a document and a query term
    
    :param document_id: the document id
    :param query_id: the tokenized query id
    """
    
    query_vector = summed_queries[query_id] / len(tokenized_queries[qid])
    doc_vector = summed_docs[document_id] / document_lengths[document_id]
    
    
    cosine_similarity = np.dot(query_vector, doc_vector)/(np.linalg.norm(query_vector)* np.linalg.norm(doc_vector))
        
    return cosine_similarity

In [ ]:
start_time = time.time()

run_w2v_retrieval('retrievals/averaged_vectors',averaged_vectors)

print ('elapsed time:',time.time()-start_time)

In [ ]:
# trec_eval command for evalutaion of averaged vectors
!trec_eval -m all_trec ./ap_88_89/qrel_validation retrievals/averaged_vectors.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"
print('\n')

# trec_eval command for test of averaged vectors
!trec_eval -m all_trec ./ap_88_89/qrel_test retrievals/averaged_vectors.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"

## Word Movers Distance

In [8]:
def wmd_vectors(document_id, query_id):
    """
    Scoring function for a document and a query term
    
    :param document_id: the document id
    :param query_id: the tokenized query id
    """
    
    query = []
    for query_term_id in tokenized_queries[query_id]: #for each word_id in query
        term = dictionary[query_term_id]
        query.append(term)
            
    document = []
    
    ext_doc_id, doc_token_ids = index.document(document_id)
    for token_id in doc_token_ids: 
        if token_id > 0:
            term = dictionary[query_term_id]
            document.append(term)
        
    distance = model.wv.wmdistance(query, document)
            
    return distance

In [9]:
start_time = time.time()

run_w2v_retrieval('retrievals/wmd_vectors',wmd_vectors)

print ('elapsed time:',time.time()-start_time)

Retrieving using retrievals/wmd_vectors


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `wmdistance` (Method will be removed in 4.0.0, use self.wv.wmdistance() instead).


elapsed time: 2763.627590894699


In [13]:
# trec_eval command for evalutaion of averaged vectors
!trec_eval -m all_trec ./ap_88_89/qrel_validation retrievals/wmd_vectors.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"
print('\n')

# trec_eval command for test of averaged vectors
!trec_eval -m all_trec ./ap_88_89/qrel_test retrievals/wmd_vectors.run | grep -E "^ndcg_cut_10\s|^map_cut_1000\s|^P_5\s|^recall_1000\s"

P_5                   	all	0.1867
recall_1000           	all	0.5079
ndcg_cut_10           	all	0.1859
map_cut_1000          	all	0.1215


P_5                   	all	0.2483
recall_1000           	all	0.4965
ndcg_cut_10           	all	0.2416
map_cut_1000          	all	0.1180


## K-means clustering

In [ ]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

start_time = time.time()

## Define vector and clusters
vectors = model.wv.syn0
clusters = 50

## Initialize clustering
kmeans_clustering = KMeans(n_clusters = clusters)
result = kmeans_clustering.fit_predict(vectors)

word_centroid_map = dict(zip( model.wv.index2word, result ))
print('Elapsed time:', time.time()-start_time)

In [ ]:
list(word_centroid_map.items())

In [ ]:
start_time = time.time()
for cluster in range(0,100):
    words = []

    for word, cluster_idx in word_centroid_map.items():
        if(cluster_idx == cluster ):
            words.append(word)
    print('Cluster ', cluster, ' amount of words: ', len(words))
    
print('Elapsed time:', time.time()-start_time)

In [ ]:
plt.figure(1)
plt.clf()
plt.imshow(result, interpolation='nearest',
          extent=(xx.min(), xx.max(), yy.min(), yy.max()),
          cmap=plt.cm.Paired,
          aspect='auto', origin='lower')

plt.plot(hpc[:, 0], hpc[:, 1], 'k.', markersize=4)
centroids = k_means.cluster_centers_
inert = k_means.inertia_
plt.scatter(centroids[:, 0], centroids[:, 1],
           marker='x', s=169, linewidths=3,
           color='w', zorder=8)
plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)
plt.xticks(())
plt.yticks(())
plt.show()

### Task 4: Learning to rank (LTR) [10 points] ###

In this task you will get an introduction into learning to rank for information retrieval, in particular pointwise learning to rank.

You will experiment with a pointwise learning to rank method, logistic regression, implemented in [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).
Train your LTR model using 10-fold cross validation on the test set.

You can explore different ways for devising features for the model. Obviously, you can use the retrieval methods you implemented in Task 1 and Task 2 as features. Think about other features you can use (e.g. query/document length). 
One idea is to also explore external sources such as Wikipedia entities (?). Creativity on devising new features and providing motivation for them will be taken into account when grading.

For every query, first create a document candidate set using the top-1000 documents using TF-IDF, and subsequently compute features given a query and a document. Note that the feature values of different retrieval methods are likely to be distributed differently.

In [104]:
#create inverse mapping, because pyndri sucks
extDocId2id={}
for ext_id in range(index.document_base(), index.maximum_document()):
    extDocId2id[index.ext_document_id(ext_id)] = ext_id
    
print(extDocId2id['AP890608-0286'])

76503


In [182]:
from collections import defaultdict

def run_ltr_retrieval(score_fn, ext_id, qid, **args):
    """
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.
    
    :param model_name: the name of the model (a string)
    :param score_fn: the scoring function (a function - see below for an example) 
    """
    retrieval_start_time = time.time()
    
    int_document_id = extDocId2id[ext_id]
    qid = str(int(qid))

    score = 0
        #hack for BM25 (iterate duplicate terms in query once)
    try:
        if score_fn==bm25:
            tokenized_queries[qid] = list(set(tokenized_queries[qid]))
    except:
        pass
    
    for query_term_id in tokenized_queries[qid]: #for each word_id in query
        if int_document_id in inverted_index[query_term_id].keys():
            document_term_freq = inverted_index[query_term_id][int_document_id]
        
            #for each document where the word appears, calculate scoring function increment its score
            new_score = score_fn(int_document_id, query_term_id, document_term_freq, **args)
            score += new_score
    
    if np.isinf(score):
        score = 0

    return score


In [179]:
## Read test file
path = "./ap_88_89/qrel_test"

columns = ['query', '?', 'doc', 'result']

trainData = pd.read_csv(path, delimiter=' ', header = None)
trainData.columns = columns
trainData = trainData.drop(columns=['?'])

ne = ['AP900109-0232',
'AP900214-0247',
'AP900215-0091',
'AP900327-0168',
'AP900427-0031',
'AP900429-0093',
'AP900607-0165',
'AP900904-0175',
'AP900913-0007',
'AP901121-0246']

# Delete non existing
trainData = trainData[~np.in1d(trainData.doc, ne)]

In [183]:
## Append TF-IDF data
trainData["tfidf"] = np.nan
for idx, row in trainData.iterrows():
    
    score = run_ltr_retrieval(tfidf, trainData.iloc[idx]['doc'], trainData.iloc[idx]['query'])
    trainData.set_value(idx, 'tfidf', score)

    
trainData
    

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


IndexError: single positional indexer is out-of-bounds

In [181]:
trainData

,query,doc,result,tfidf
0,68,AP880218-0195,1,24.020823
1,68,AP880229-0184,1,23.671856
2,68,AP880301-0033,1,35.006768
3,68,AP880302-0095,0,0.000000
4,68,AP880312-0116,1,0.000000
5,68,AP880312-0142,0,43.604136
6,68,AP880314-0106,1,0.000000
7,68,AP880318-0316,0,13.595374
8,68,AP880321-0194,1,33.587524
9,68,AP880321-0278,0,45.292147


### Task 4: Write a report [20 points; instant FAIL if not provided] ###

The report should be a PDF file created using the [sigconf ACM template](https://www.acm.org/publications/proceedings-template) and will determine a significant part of your grade.

   * It should explain what you have implemented, motivate your experiments and detail what you expect to learn from them. **[10 points]**
   * Lastly, provide a convincing analysis of your results and conclude the report accordingly. **[10 points]**
      * Do all methods perform similarly on all queries? Why?
      * Is there a single retrieval model that outperforms all other retrieval models (i.e., silver bullet)?
      * ...

**Hand in the report and your self-contained implementation source files.** Only send us the files that matter, organized in a well-documented zip/tgz file with clear instructions on how to reproduce your results. That is, we want to be able to regenerate all your results with minimal effort. You can assume that the index and ground-truth information is present in the same file structure as the one we have provided.
